## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [3]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [8]:
import urllib.request
import zipfile
import lxml.etree

In [6]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [9]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [10]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [11]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [12]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [13]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [14]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [15]:
len(sentences_ted)

266694

In [16]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [45]:
# ...
#type(sentences_ted)
#type(sentences_ted[0])
#word occurences
words = {} #empty dict
for s in sentences_ted: #iterate through the sentences
    for w in s: #iterate through the words
        if w not in words:
            words[w] = 0
        words[w] += 1 #increment occurency of word

#sanity check the words dict
#print(words)
#Now sort the list for its highest values
result = sorted(words, key=lambda k: words[k], reverse=True)
print(result[:100])
#Get the counts from the most common keys with list comprehension
counts = [words[i] for i in result]
print(counts[:100])
#Take only the first 1000 words in the list
counts_ted_top1000 = counts[:1000]
print('Finished processing')

['the', 'and', 'to', 'of', 'a', 'that', 'i', 'in', 'it', 'you', 'we', 'is', 's', 'this', 'so', 'they', 'was', 'for', 'are', 'have', 'but', 'what', 'on', 'with', 'can', 't', 'about', 'there', 'be', 'as', 'at', 'all', 'not', 'do', 'my', 'one', 're', 'people', 'like', 'if', 'from', 'now', 'our', 'he', 'an', 'just', 'these', 'or', 'when', 'because', 'very', 'me', 'out', 'by', 'them', 'how', 'know', 'up', 'going', 'had', 'more', 'think', 'who', 'were', 'see', 'your', 'their', 'which', 'would', 'here', 'really', 'get', 've', 'then', 'm', 'world', 'us', 'time', 'some', 'has', 'don', 'actually', 'into', 'way', 'where', 'will', 'years', 'things', 'other', 'no', 'could', 'go', 'well', 'want', 'been', 'make', 'right', 'she', 'said', 'something']
[207748, 149305, 125169, 114818, 105399, 95146, 83180, 78070, 74738, 70923, 67629, 63251, 57156, 49241, 37014, 33102, 30806, 29713, 27995, 27344, 26732, 26519, 25962, 24706, 23377, 22757, 21246, 21041, 20201, 19488, 19216, 19021, 18626, 17928, 17908, 1755

Plot distribution of top-1000 words

In [46]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [48]:
from gensim.models import Word2Vec

In [49]:
# min_count changed to 10 as specified in assignment
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=10, workers=4)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [50]:
model_ted.most_similar("man")

[('woman', 0.8539605736732483),
 ('guy', 0.8160413503646851),
 ('lady', 0.7701655030250549),
 ('boy', 0.7430141568183899),
 ('girl', 0.7327548861503601),
 ('soldier', 0.7306192517280579),
 ('gentleman', 0.7260057926177979),
 ('kid', 0.7019530534744263),
 ('poet', 0.6973050236701965),
 ('writer', 0.6719167232513428)]

In [51]:
model_ted.most_similar("computer")

[('machine', 0.7681055068969727),
 ('software', 0.7270140051841736),
 ('robot', 0.6968362331390381),
 ('3d', 0.6706424951553345),
 ('program', 0.6486856937408447),
 ('simulation', 0.6476264595985413),
 ('device', 0.6470869779586792),
 ('interface', 0.6418406367301941),
 ('video', 0.6415716409683228),
 ('camera', 0.6378358006477356)]

In [62]:
print(model_ted.most_similar("gym")) #I guess compost and nazis have lots to do with the gym
print(model_ted.most_similar("scientist")) #Apparently scientists resemble designers
print(model_ted.most_similar("german")) #Wow, ok! Gandhi ...

[('compost', 0.8236953020095825), ('nazis', 0.802388608455658), ('shorts', 0.7780681252479553), ('glider', 0.7771477699279785), ('toyota', 0.7748559713363647), ('troop', 0.7732051610946655), ('razor', 0.7715463638305664), ('auschwitz', 0.7711904644966125), ('booth', 0.7703593969345093), ('ruins', 0.7699561715126038)]
[('designer', 0.832504391670227), ('writer', 0.8192892074584961), ('physicist', 0.816719651222229), ('researcher', 0.78293776512146), ('musician', 0.7826260924339294), ('psychologist', 0.7770742177963257), ('journalist', 0.7438817024230957), ('filmmaker', 0.7324694991111755), ('professor', 0.7267361879348755), ('mathematician', 0.7163704037666321)]
[('french', 0.8500941395759583), ('gandhi', 0.8142837285995483), ('british', 0.7985963821411133), ('japanese', 0.798580527305603), ('dutch', 0.786578893661499), ('sierra', 0.7822566032409668), ('nazi', 0.7759741544723511), ('palestinian', 0.7688199877738953), ('russian', 0.7676201462745667), ('robert', 0.7670885324478149)]


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [ ]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [ ]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [ ]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [ ]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [ ]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [ ]:
# ...

#### t-SNE visualization

In [ ]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)